# **Riammissione Pazienti Diabetici**

Il numero di riammissioni ospedaliere per alcune patologie viene considerato un importante indicatore di qualità di un ospedale, oltre ad impattare in modo pesante sul budget destinato alla sanità.

A causa della privacy trovare un buon dataset è difficile: in assenza di dati reali ci siamo basati su un dataset pubblico, reperibile al link https://archive.ics.uci.edu/ml/datasets/diabetes+130-us+hospitals+for+years+1999-2008# e che copre i dati di ricovero relativi a pazienti diabetici in 130 ospedali statunitensi, durante un periodo di 10 anni (1999 - 2008).

## **Descrizione del Dataset**
Sono registrati 101766 ricoveri unici, fatti da 71518 pazienti differenti, con feature di diverso tipo:
* identificativo del ricovero
* identificativo del paziente
* dati anagrafici (sesso, fascia di età, peso)
* tipo di ammissione / dimissione 
* tempo di permanenza in ospedale
* codice legato all'assicurazione medica (negli Stati Uniti i pazienti possono accedere alle cure solo se iscritti a compagnie assicurative)
* reparto di ammissione
* numero di esami / operazioni / medicazioni effettuati sul paziente
* numero di visite al pronto soccorso / visite specialistiche / ricoveri nell'anno precedente
* numero di diagnosi effettuate sul paziente durante il ricovero
* codice identificativo delle prime tre diagnosi secondo il codice ICD    https://en.wikipedia.org/wiki/International_Classification_of_Diseases 
* tipo di reparto dove è avvenuto il ricovero
* terapia farmacologica, viene indicato se un farmaco è stato usato e come il dosaggio è cambiato nel tempo (stabile, crescente, decrescente)
* cambio di terapia diabetica (dosaggio o farmaco)
* utilizzo di terapia diabetica
* valori glicemia nel sangue, divisi in range ( > 200, > 300, Normale) ed eventualmente "None" se l'esame non è stato effettuato
* valori di emoglobina glicata (A1C) al momento dell'ammissione, divisi per range ( > 7 %, > 8 %, Normale) ed eventualmente "None" se l'esame non è stato effettuato

**Il campo più importante è quello che indica se il paziente è stato nuovamente ricoverato entro 30 giorni, oltre i 30 giorni o se non è mai stato riammesso in ospedale: sarà oggetto del nostro modello predittivo.**

In [ ]:
# CARICO LE LIBRERIE NECESSARIE PER FAR FUNZIONARE IL CODICE, E IL DATASET UTILIZZATO NELLA DEMO
import pandas as pd
from IPython.display import display, HTML

original = pd.read_csv('demo_dataset.csv')

In [ ]:
# UTILIZZIAMO UN ESTRATTO DEL DATASET DEMO PER VISUALIZZARE I DATI "NUDI E CRUDI": LE CONSIDERAZIONI CHE FAREMO SONO QUELLE CHE SONO STATE UTILIZZATE PER CREARE IL DATASET "PULITO" SU CUI ABBIAMO FATTO IL TRAINING DEL MODELLO
samples = original.sample(10).sort_values(by = 'patient_nbr')
display(HTML(samples.to_html()))

## **Preprocessing dei dati**
###  1. Pulizia e Codifica
Alla base di un buon modello di ML si trovano dei dati puliti, il che significa levare le righe con poche features valorizzate, rimuovere colonne in cui la maggior parte dei dati è mancante, oppure utilizzare degli aggregati al posto dei dati "nudi e crudi".

**STEP 0: STUDIO DELLA VARIABILE DI OUTPUT**
La variabile target del nostro modello, ***READMISSION***, presenta 3 valori differenti in base al fatto che una riammissione sia avvenuta entro 30 giorni dalla precedente, oltre i 30 giorni o mai: per semplificare il problema si terrà conto solo della riammissione entro 30 giorni (codificata a 1), negli altri due casi varrà 0.

**STEP 1: COSA FARE CON I VALORI MANCANTI**
Queste le operazioni che vengono fatte nella prima fase di preprocessing, che riguarda la pulizia dei campi con pochi dati:
* viene cancellata la colonna del peso, perchè manca il 98% dei dati
* vengono cancellate le colonne relative all'assicurazione e al reparto dove avviene il ricovero (40% - 50%)
* vengono rimosse le colonne relative alla seconda e alla terza diagnosi
* infine vengono rimosse le colonne che presentano un unico valore, citogliptone ed examide

Alla pulizia delle colonne aggiungiamo la pulizia delle righe, in particolare:
* Righe in cui non compare il sesso del paziente
* Righe che riportano il decesso del paziente (discharge_disposition_id = 11)

**STEP 2: CODIFICA ETÀ**
L'età dei pazienti è stata registrata in fasce, del tipo \[10-20)\] (compresa fra i 10 e 20 anni esclusi): poichè l'età è da considerarsi una variabile continua e non categorica, si è scelto di prendere come valore numerica l'età media di ogni fascia (quindi nel caso di cui sopra 15 anni per esempio)

**STEP 3: RIDUZIONE NUMERO DI CATEGORIE**
Molte colonne presentano valori categorici, che possono complicare enormemente la fase di modelling se il numero di label per una determinata feature è grande: un esempio la riduzione del numero di codici di ammissione (tipo e gravità) e di dimissione, che portano ad avere
* Gravità di Ammissione --> da 8 a 4 codici 
* Tipi di Ammissione --> da 26 a 6
* Codici di Dimissione --> da 29 a 10 codici

Un'altra feature che presenta molti valori categorici è la diagnosi, con ben 717 valori unici! La mappatura in questo caso porta ad avere solamente 9 valori categorici differenti, che raggruppano le diagnosi per specialità (malattie dell'apparato circolatorio, respiratorio, digerente...).

**STEP 4: CAMBI DI TERAPIA DEI MEDICINALI**
Contiamo il numero di medicinali la cui posologia viene cambiata durante il ricovero, per ottenere la colonna "numchange" (ogni medicinale che presenta valori "UP" o "DOWN" viene codificato a 1).

**STEP 5: CODIFICA DELL'UTILIZZO DEI MEDICINALI**
Abbiamo deciso, per semplificare il problema, di adottare la regola che se un medicinale viene utilizzato viene codificato con un 1, a prescindere dal piano terapeutico (dose stabile, aumentata, diminuita).

**STEP 6: CODIFICA DEI RISULTATI DELLE ANALISI DIABETICHE**
I risultati degli esami per l'emoglobina glicata (A1Cresult) e per la glicemia nel sangue hanno ambedue 3 valori differenti (normale, superiore a un primo livello di soglia, superiore a un secondo livello di soglia) più uno aggiuntivo per indicare che l'esame non è stato effettuato: mentre i valori anormali vengono unificati a un solo livello (1), valori normali vengono identificati da uno 0 mentre esame non effettuato dal codice -99.

**STEP 7: INCONTRI MULTIPLI**
Per un determinato paziente, se sono presenti ricoveri multipli viene studiato solamente il primo e tutti gli altri vengono eliminati (studiare l'ultimo ricovero porterebbe a uno sbilanciamento delle etichette riammissione/non riammissione con un ratio 96/4).

### 2. Trasformazioni Matematiche
Alla fase precedente di preprocessing si aggiunge una, più avanzata, basata sulla creazione di nuove features o sulla trasformazione di features già presenti tramite funzioni matematiche.

**STEP 1: UTILIZZO DEI SERVIZI OSPEDALIERI**
La colonna "service_utilization" somma le volte in cui un paziente ha visitato un Pronto Soccorso, è stato ricoverato o ha fatto una visita specialistica nell'anno precedente al ricovero.

**STEP 2: STUDIO DELLE DISTRIBUZIONI DEI DATI NUMERICI**
Quando si costruisce un modello a partire da dati numerici è consuetudine standardizzarli, che significa trasformare la loro distribuzione in modo che quella risultante abbia valor medio $\mathbf{\mu = 0}$ e deviazione standard $\mathbf{\sigma = 1}$ e forma **gaussiana (o normale)**: per fare questo bisogna che i dati originari abbiano loro stessi un andamento gaussiano, anche se con altri valori medi e deviazione standard.\
Una tipica distribuzione normale è simmetrica rispetto al valor medio (skew = 0) e non presenta code lunghe e nemmeno troppo corte (kurtosis = 3): se la variabile non presenta un simile andamento allora si applica una trasformazione intermedia, che rende la distribuzione **lognormale** perchè è il logaritmo dei valori che presenta queste caratteristiche.\
In particolare, viene applicata la formula $v^* = \log(v)$ quando la variabile $v$ presenta valori non nulli mentre in caso contrario viene applicata $v^* = \log(1+v)$.\
Una volta effettuata l'eventuale trasformazione, le variabili vengono standardizzate a una distribuzione normale secondo la formula
$$\mathbf{x^*} = \frac{x-\mu_x}{\sigma_x}$$
dove valori medi e sigma sono calcolati su ogni singola variabile in gioco.

L'analisi delle distribuzioni delle variabili numeriche evidenzia che il numero di ricoveri (number_inpatient), il numero di visite al pronto soccorso (number_emergency), il numero di visite specialistiche (number_outpatient) e di conseguenza il numero di volte che il paziente è stato assistito (service_utilization) richiedono una trasformazione $v^* = \log(1+v)$ prima di poter attuare la standardizzazione dei valori.

**STEP 3: STANDARDIZZAZIONE DEI VALORI**
Dopo aver reso "gaussiane" le distribuzioni è possibile standardizzare le variabili, calcolando deviazioni standard $\sigma$ e valori medi $\mu$ (**Tabella 1**)

| **Feature**               | $\sigma$    | $\mu$       |
|---------------------------|-------------|-------------|
| num_medications           | 8.092612416 | 15.98155214 |
| time_in_hospital          | 2.974527815 | 4.389452657 |
| number_inpatient_log1p    | 0.510433232 | 0.324836712 |
| num_procedures            | 1.70033455  | 1.330803036 |
| numchange                 | 0.487858032 | 0.287544946 |
| num_lab_procedures        | 19.62022838 | 42.94378746 |
| number_outpatient_log1p   | 0.429393649 | 0.172339628 |
| number_diagnoses          | 1.938210581 | 7.409338793 |
| age                       | 15.94749609 | 65.83180184 |
| number_emergency_log1p    | 0.315405297 | 0.101963135 |
| nummed                    | 0.921620211 | 1.185537355 |
| service_utilization_log1p | 0.66237317  | 0.513654407 |


### 3. Creazione Nuove Variabili
L'ultima fase di preprocessing comporta la creazione di nuove variabili, utili per la modellazione.\
**STEP 1: VARIABILI DI INTERAZIONE**
Le variabili di interazione possono aumentare di molto la capacità predittiva di un modello.
L'interazione fra variabili viene studiata usando il coeffiente di correlazione di Pearson, che misura quanto una variabile cambia in relazione al cambiamento di un'altra: valori vicini a 1 indicano correlazione fra variabili positiva (se aumenta una variabile aumenta anche l'altra), valori vicini a -1 indicano invece correlazione negativa (quando aumenta una variabile l'altra diminuisce).
A partire da uno studio sulle correlazioni di coppie di variabili, vengono create delle variabili aggiuntive che poi possono essere usate per il modello: il loro valore viene calcolato, riga per riga, moltiplicando i valori delle colonne indicate.
Troviamo perciò 
* num_medications X time_in_hospital
* num_medications X num_procedures 
* time_in_hospital X num_lab_procedures
* num_medications X num_lab_procedures
* num_medications X number_diagnoses
* age X number_diagnoses
* change X num_medications
* number_diagnoses X time_in_hospital
* num_medications X numchange

**STEP 2: VARIABILI DUMMY**\
Le variabili dummy nascono da una binarizzazione delle variabili categoriche, con un metodo detto "one-hot-encoding" che trasforma ogni livello in una variabile differente (per esempio, la colonna ETNIA "esplode" in 5 colonne differenti del tipo "ETNIA_Caucasico", "ETNIA_AfroAmericano", "ETNIA_Asiatico") valorizzata in modo binario (per esempio "ETNIA_AfroAmericano" = 1, tutte le altre colonne per l'etnia poste a 0).
Ecco l'elenco delle colonne originarie e di quelle dummy:

\['race'] $\longrightarrow$ \['race_?', 'race_AfricanAmerican', 'race_Asian', 'race_Caucasian', 'race_Hispanic', 'race_Other']\
\['gender'] $\longrightarrow$ ['gender_0', 'gender_1', 'gender_Unknown/Invalid']\
\['admission_type_id'] $\longrightarrow$ ['admission_type_id_1', 'admission_type_id_3', 'admission_type_id_4', 'admission_type_id_5']\
\['discharge_disposition_id'] $\longrightarrow$ ['discharge_disposition_id_1', 'discharge_disposition_id_2', 
'discharge_disposition_id_7',
       'discharge_disposition_id_10', 'discharge_disposition_id_11', 'discharge_disposition_id_18',
       'discharge_disposition_id_19', 'discharge_disposition_id_20',
       'discharge_disposition_id_27', 'discharge_disposition_id_28']\
\['admission_source_id'] $\longrightarrow$ ['admission_source_id_1', 'admission_source_id_4', 'admission_source_id_7',
       'admission_source_id_8', 'admission_source_id_9',
       'admission_source_id_11']\
\['max_glu_serum'] $\longrightarrow$ ['max_glu_serum_-99', 'max_glu_serum_0', 'max_glu_serum_1']\
\['A1Cresult'] $\longrightarrow$ ['A1Cresult_-99', 'A1Cresult_0', 'A1Cresult_1']\
\['level1_diag1'] $\longrightarrow$ ['level1_diag1_0.0', 'level1_diag1_1.0', 'level1_diag1_2.0',
       'level1_diag1_3.0', 'level1_diag1_4.0', 'level1_diag1_5.0',
       'level1_diag1_6.0', 'level1_diag1_7.0', 'level1_diag1_8.0']

### 4. SMOTE
L'ultima fase di preprocessing riguarda il bilanciamento dei dati: al momento il dataset infatti è fortemente sbilanciato, con un rapporto fra persone non riammesse e riammesse  che vale 1/10.\
Per ovviare a questo problema è stata utilizzata la tecnica dello **SMOTE (Syntetic Minority Oversampling Technique)**, che consente di creare dati "finti" a partire dalle distribuzioni delle variabili che definiscono i dati: in particolare è stato applicato ai dati che definiscono la classe rappresentata in modo minore, quella dei non riammessi.

## **Training del Modello**
Una volta processati i dati, si è passati alla fase di modelling vera e propria: vista la grande presenza di dati categorici, è stata impiegata una Random Forest su tutte le variabili di input disponibili nel dataset (eccetto l'identificativo del paziente e del ricovero).

Il modello è stato addestrato e pubblicato in Azure Machine Learning, uno strumento Microsoft per la gestione a 360 gradi di un progetto di Machine Learning.

## **Predizione delle Riammissioni**
L'algoritmo di scoring, che consente di prevedere la riammissione di un paziente diabetico entro 30 giorni dalla riammissione, viene testato su un campione casuale di ricoveri presenti nel dataset originario.
Per velocizzare il processo di previsione vengono passati all'algoritmo i dati già puliti.

In [ ]:
# GLI ID UTENTE CI AIUTERANNO A TROVARE I DATI CORRISPONDENTI NEL DATASET PULITO
ids = samples['patient_nbr'].unique()  

In [ ]:
# CARICO IL DATASET CON I DATI GIÀ PULITI
cleaned = pd.read_csv('demo_dataset_cleaned.csv')

In [ ]:
# SELEZIONO, NEL DATASET PULITO, SOLO LE RIGHE RELATIVE AGLI UTENTI CHE ABBIAMO ESTRATTO IN MODO CASUALE IN PRECEDENZA
cleaned_X = cleaned.loc[cleaned['patient_nbr'].isin(ids)]

In [ ]:
cleaned_X

In [ ]:
# ELIMINO DAI DATI LE COLONNE RELATIVE ALLA RIAMMISSIONE, L'ID PAZIENTE E IL CODICE RICOVERO
data = cleaned_X.drop(['readmitted', 'patient_nbr', 'encounter_id'], axis = 1)

In [ ]:
# CHIAMO L'ENDPOINT E FACCIO LA PREVISIONE
scoring_uri = 'http://0a392c37-9664-4df8-b6df-926993e593f3.westeurope.azurecontainer.io/score'
import requests
import json

scoring_dataset = data.values.tolist()
# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": scoring_dataset})

# Set the content type
# Set the content type
headers = { 'Content-Type':'application/json' }

results = json.loads(requests.post(scoring_uri , input_json, headers = headers).json())['data']

for i in range(0, len(scoring_dataset)):
    flag = ['NON RIAMMESSO' if results['predictions'][i] == 0 else 'RIAMMESSO'][0]
    prob = results['probabilities'][i]
    print('Paziente %s con probabilità %s' % (str(flag), str(prob)))

Data la presenza, nel dataset, di un alto numero di pazienti non riammessi entro 30 giorni dalla dimissione è molto probabile che il campione qui esaminato presenti almeno 8 risultati etichettati come "non riammesso".

-------------------------

**PER TESTARE UN ALTRO CAMPIONE DI PAZIENTI, É SUFFICIENTE ESEGUIRE IL NOTEBOOK DA CAPO: VERRÀ SCELTO, IN MODO CASUALE, UN NUOVO SET DI DATI.**